# Business case

* Predict the outcome of a baseball game based on match metrics (other than the score)
* Helping the team focus on the right skills to train for as well as chosing the right game strategy

* create 'historical' model then deploy it for a team specifically


## Further work

* Deploy and fine tune model to a specific team
* Be able to identify the players that "match the model"


In [51]:
import pandas as pd
import seaborn as sns

In [52]:
df = pd.read_csv('MLB-GameLogs-1871_2016.csv')
df

/home/antoine/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (13,14,19,85,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,19460416,0,Tue,SLA,AL,1,DET,AL,1,1,...,Pinky Higgins,5.0,richp101,Paul Richards,2.0,newhh101,Hal Newhouser,1.0,NaN,Y
1,19460416,0,Tue,NYA,AL,1,PHA,AL,1,5,...,George Kell,5.0,handg101,Gene Handley,4.0,chrir101,Russ Christopher,1.0,NaN,Y
2,19460416,0,Tue,BOS,AL,1,WS1,AL,1,6,...,Jerry Priddy,4.0,evana101,Al Evans,2.0,wolfr102,Roger Wolff,1.0,NaN,Y
3,19460416,0,Tue,CLE,AL,1,CHA,AL,1,1,...,Thurman Tucker,8.0,tresm101,Mike Tresh,2.0,dietb101,Bill Dietrich,1.0,NaN,Y
4,19460416,0,Tue,BRO,NL,1,BSN,NL,1,3,...,Skippy Roberge,5.0,culld101,Dick Culler,6.0,sainj101,Johnny Sain,1.0,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94927,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,...,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
94928,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,...,Andres Blanco,6.0,altha001,Aaron Altherr,9.0,eickj001,Jerad Eickhoff,1.0,NaN,Y
94929,20161002,0,Sun,LAN,NL,162,SFN,NL,162,1,...,Joe Panik,4.0,gillc001,Conor Gillaspie,5.0,moorm003,Matt Moore,1.0,NaN,Y
94930,20161002,0,Sun,PIT,NL,162,SLN,NL,162,4,...,Jedd Gyorko,4.0,gricr001,Randal Grichuk,8.0,waina001,Adam Wainwright,1.0,NaN,Y


# Data exploration

Let's look at the teams:

In [53]:
visiting = set(df['v_name'])
home = set(df['h_name'])
print(len(visiting), visiting, '\n\n', len(home), home)

44 {'SDN', 'NYN', 'KCA', 'PHI', 'CHA', 'SLN', 'FLO', 'CIN', 'CLE', 'NY1', 'HOU', 'ARI', 'WAS', 'SLA', 'KC1', 'LAA', 'MLN', 'BSN', 'COL', 'BAL', 'PIT', 'MIN', 'PHA', 'WS2', 'TBA', 'MIA', 'BRO', 'BOS', 'MON', 'LAN', 'SEA', 'SE1', 'CHN', 'MIL', 'OAK', 'SFN', 'CAL', 'DET', 'WS1', 'TOR', 'ANA', 'NYA', 'TEX', 'ATL'} 

 44 {'SDN', 'NYN', 'KCA', 'PHI', 'CHA', 'SLN', 'FLO', 'CLE', 'CIN', 'NY1', 'HOU', 'ARI', 'WAS', 'SLA', 'KC1', 'LAA', 'MLN', 'BSN', 'COL', 'BAL', 'PIT', 'MIN', 'PHA', 'WS2', 'TBA', 'MIA', 'BRO', 'BOS', 'MON', 'LAN', 'SEA', 'SE1', 'CHN', 'MIL', 'OAK', 'SFN', 'CAL', 'DET', 'WS1', 'TOR', 'ANA', 'NYA', 'TEX', 'ATL'}


In [58]:
# Dropping unecessary columns
df = df.drop(columns=['acquisition_info','additional_info'])

# Dropping redundant info
dropping = []
for column in df.columns[7:]:
    if column.endswith('_def_pos'):
        dropping.append(column)
    elif '_name' in column:
        dropping.append(column)
df = df.drop(columns=dropping)

# Removing the 2 forfeited games
df = df[df['forefeit'].isna()==True]

# Removing interrupted games
df = df[df['completion'].isna()==True]

In [80]:
df['park_id'].value_counts()

CHI11    4059
BOS07    4049
NYC16    3394
LOS03    3135
DET04    2687
         ... 
JER02      15
TOK01       8
LBV01       6
SYD01       2
FTB01       1
Name: park_id, Length: 75, dtype: int64

In [59]:
for column in df.columns:
    print(column)

date
number_of_game
day_of_week
v_name
v_league
v_game_number
h_name
h_league
h_game_number
v_score
h_score
length_outs
day_night
completion
forefeit
protest
park_id
attendance
length_minutes
v_line_score
h_line_score
v_at_bats
v_hits
v_doubles
v_triples
v_homeruns
v_rbi
v_sacrifice_hits
v_sacrifice_flies
v_hit_by_pitch
v_walks
v_intentional walks
v_strikeouts
v_stolen_bases
v_caught_stealing
v_grounded_into_double
v_first_catcher_interference
v_left_on_base
v_pitchers_used
v_individual_earned_runs
v_team_earned_runs
v_wild_pitches
v_balks
v_putouts
v_assists
v_errors
v_passed_balls
v_double_plays
v_triple_plays
h_at_bats
h_hits
h_doubles
h_triples
h_homeruns
h_rbi
h_sacrifice_hits
h_sacrifice_flies
h_hit_by_pitch
h_walks
h_intentional walks
h_strikeouts
h_stolen_bases
h_caught_stealing
h_grounded_into_double
h_first_catcher_interference
h_left_on_base
h_pitchers_used
h_individual_earned_runs
h_team_earned_runs
h_wild_pitches
h_balks
h_putouts
h_assists
h_errors
h_passed_balls
h_double

In [40]:
print(dropping)

['hp_umpire_name', '1b_umpire_name', '2b_umpire_name', '3b_umpire_name', 'lf_umpire_name', 'rf_umpire_name', 'v_manager_name', 'h_manager_name', 'winning_pitcher_name', 'losing_pitcher_name', 'saving_pitcher_name', 'winning_rbi_batter_id_name', 'v_starting_pitcher_name', 'h_starting_pitcher_name', 'v_player_1_name', 'v_player_1_def_pos', 'v_player_2_name', 'v_player_2_def_pos', 'v_player_3_name', 'v_player_3_def_pos', 'v_player_4_name', 'v_player_4_def_pos', 'v_player_5_name', 'v_player_5_def_pos', 'v_player_6_name', 'v_player_6_def_pos', 'v_player_7_name', 'v_player_7_def_pos', 'v_player_8_name', 'v_player_8_def_pos', 'v_player_9_name', 'v_player_9_def_pos', 'h_player_1_name', 'h_player_1_def_pos', 'h_player_2_name', 'h_player_2_def_pos', 'h_player_3_name', 'h_player_3_def_pos', 'h_player_4_name', 'h_player_4_def_pos', 'h_player_5_name', 'h_player_5_def_pos', 'h_player_6_name', 'h_player_6_def_pos', 'h_player_7_name', 'h_player_7_def_pos', 'h_player_8_name', 'h_player_8_def_pos', 'h_p

In [32]:
df['h_player_1_def_pos']

0        6.0
1        7.0
2        5.0
3        9.0
4        4.0
        ... 
94927    8.0
94928    4.0
94929    8.0
94930    3.0
94931    8.0
Name: h_player_1_def_pos, Length: 94932, dtype: float64

In [31]:
df[df['winning_pitcher_id']==df['h_starting_pitcher_id']]

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,19460416,0,Tue,SLA,AL,1,DET,AL,1,1,...,Pinky Higgins,5.0,richp101,Paul Richards,2.0,newhh101,Hal Newhouser,1.0,NaN,Y
4,19460416,0,Tue,BRO,NL,1,BSN,NL,1,3,...,Skippy Roberge,5.0,culld101,Dick Culler,6.0,sainj101,Johnny Sain,1.0,NaN,Y
6,19460416,0,Tue,PHI,NL,1,NY1,NL,1,4,...,Sid Gordon,7.0,kerrb101,Buddy Kerr,5.0,voisb101,Bill Voiselle,1.0,NaN,Y
10,19460417,0,Wed,NYA,AL,2,PHA,AL,2,1,...,George Kell,5.0,handg101,Gene Handley,4.0,newsb101,Bobo Newsom,1.0,NaN,Y
14,19460417,0,Wed,PHI,NL,2,NY1,NL,2,2,...,Willard Marshall,7.0,kerrb101,Buddy Kerr,5.0,joycb102,Bob Joyce,1.0,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94916,20161001,0,Sat,MIA,NL,160,WAS,NL,161,1,...,Pedro Severino,2.0,espid001,Danny Espinosa,6.0,roart001,Tanner Roark,1.0,NaN,Y
94917,20161002,0,Sun,HOU,AL,162,ANA,AL,162,1,...,Rafael Ortega,8.0,perec003,Carlos Perez,2.0,pennc001,Cliff Pennington,4.0,NaN,Y
94925,20161002,0,Sun,DET,AL,161,ATL,NL,161,0,...,Jace Peterson,4.0,swand001,Dansby Swanson,6.0,tehej001,Julio Teheran,1.0,NaN,Y
94929,20161002,0,Sun,LAN,NL,162,SFN,NL,162,1,...,Joe Panik,4.0,gillc001,Conor Gillaspie,5.0,moorm003,Matt Moore,1.0,NaN,Y


In [26]:
diff = df['v_score']-df['h_score']
len(diff[diff==0])  # 162
len(diff)  # 94932

94932

The 70's and 80's are missing from the dataset but we still have access to 50 years of data:

In [10]:
year = []
for d in df['date']:
    year.append(str(d)[:4])
print(len(set(year)))
set(sorted(year))

51


{'1946',
 '1947',
 '1948',
 '1949',
 '1950',
 '1951',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016'}

# Feature selection

Look at the vif and run a decision tree to find the feature importance to help select those.

In [ ]:
y = df['winner']
X = df.drop(columns=['winner'])

## Correlation matrix

In [ ]:
sns.heatmap(X.corr(), center=0, annot=True);

## Variance Inflation Factor

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(X.columns, vif))

## Decision Tree to identify important features

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Image
from pydotplus import graph_from_dot_data

from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42
                                                   )
# Initialising K-Folds
kfold = KFold(n_splits=5,
              random_state=42,
              shuffle=True
             )

In [ ]:
# OneHotEncode step
ohe = OneHotEncoder()
ohe.fit(X_train)
X_train_ohe = ohe.transform(X_train).toarray()

In [ ]:
# Train Decision tree
clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(X_train_ohe, y_train)

In [ ]:
# Graphical representation of the decision tree
ohe_df = pd.DataFrame(X_train_ohe,
                      columns=ohe.get_feature_names(X_train.columns)
                     )
dot_data = export_graphviz(clf,
                           out_file=None, 
                           feature_names=ohe_df.columns,  
                           class_names=np.unique(y).astype('str'), 
                           filled=True,
                           rounded=True,
                           special_characters=True
                          )
graph = graph_from_dot_data(dot_data)  
Image(graph.create_png())

# Feature importance
sns.barplot(x=ohe_df.columns,
            # x=clf.estimator_,
            y=clf.feature_importances_
           )

In [ ]:
# Evaluate the precision of the classifier
X_test_ohe = ohe.transform(X_test)
y_preds = clf.predict(X_test_ohe)

print('Accuracy: ', accuracy_score(y_test, y_preds))

# Baseline model: logistic regression

## Scaling the data

In [ ]:
# Getting dummies /!\ change df[relevant_columns] with the columns that need to get dumbed down
X = pd.get_dummies(df[relevant_columns], drop_first=True)
X = X.fillna(value=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# Scaling
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')
model_log = logreg.fit(X_train, y_train)
model_log

### Accuracy of the logistic model

In [ ]:
y_hat_test = logreg.predict(X_test)
y_hat_train = logreg.predict(X_train)
residuals = np.abs(y_train - y_hat_train)
print(pd.Series(residuals).value_counts(normalize=True))